In [37]:
from PIL import Image
from PIL.ExifTags import TAGS
import os
from datetime import datetime
from pillow_heif import register_heif_opener
register_heif_opener()


def get_image_date(image_path):
    """Extracts a date of picture creation from EXIF metadata."""
    try:
        with Image.open(image_path) as img:
            exif_data = img._getexif()
            if exif_data:
                for tag, value in exif_data.items():
                    tag_name = TAGS.get(tag, tag)
                    if tag_name == "DateTime":
                        return value
    except Exception as e:
        print(f"Error reading {image_path}: {e}")
    return "No date :-("


def format_date(date_string):
    """Formats the date as IMG_[YYYY][MM][DD]_[HH][MM][SS]."""
    try:
        dt = datetime.strptime(date_string, "%Y:%m:%d %H:%M:%S")
        return dt.strftime("IMG_%Y%m%d_%H%M%S")
    except ValueError:
        return "Incorrect date"


def list_images_with_dates(directory):
    """Lists all imagesin dir with their creation dates in IMG_[Y][M][D]_[H][M][S] format."""
    normal_extensions = ('.jpg', '.jpeg', '.png', '.tiff')
    
    for filename in os.listdir(directory):
        if filename.lower().endswith(normal_extensions):
            image_path = os.path.join(directory, filename)
            date_created = get_image_date(image_path)
            formatted_date = format_date(date_created) if date_created != "No date :-(" else date_created
            print(f"{filename}: {formatted_date}")
        elif filename.lower().endswith(".heic"):
            image_path = os.path.join(directory, filename)
            img = Image.open(image_path)
            exif_data = img.getexif()
            for tag, value in exif_data.items():
                if tag == 306:
                    # print(value)
                    print(format_date(value))
            # print(f"HEIC!!: {image_path}")

def rename_images(directory):
    """Renames images in a directory based on their capture date."""
    normal_extensions = ('.jpg', '.jpeg', '.png', '.tiff')
    
    for filename in os.listdir(directory):
        if filename.lower().endswith(normal_extensions):
            image_path = os.path.join(directory, filename)
            date_created = get_image_date(image_path)
            
            formatted_date = format_date(date_created)
            new_filename = formatted_date + os.path.splitext(filename)[1]
            new_path = os.path.join(directory, new_filename)
            os.rename(image_path, new_path)
            print(f"Renamed: {filename} -> {new_filename}")

        elif filename.lower().endswith(".heic"):
            image_path = os.path.join(directory, filename)
            img = Image.open(image_path)
            exif_data = img.getexif()
            for tag, value in exif_data.items():
                if tag == 306:
                    print(value)
                    formatted_date = format_date(value)
                    new_filename = formatted_date + os.path.splitext(filename)[1]
                    new_path = os.path.join(directory, new_filename)
                    os.rename(image_path, new_path)
                    print(f"Renamed: {filename} -> {new_filename}")


In [38]:
directory = r"/Users/chamster/Desktop/cadia"

In [39]:
# if os.path.exists(directory):
#     list_images_with_dates(directory)
# else:
#     print("Invalid directory!")


if os.path.exists(directory):
    rename_images(directory)
else:
    print("Invalid directory!")

2025:02:26 11:13:47
Renamed: Incorrect date.HEIC -> IMG_20250226_111347.HEIC
